# Decision Trees
---

## 00. Imports

In [1]:
import numpy as np

## 01. Gini Index

In [2]:
def group_gini_index(samples: np.array) -> float:
    n_samples = samples.shape[0]
    _, n_groups = np.unique(samples, return_counts=True)
    return 1 - sum(map(lambda x: (x/n_samples)**2, n_groups))

In [3]:
def groups_gini_index(first: np.array, second: np.array) -> float:
    n_first = first.shape[0]
    n_second = second.shape[0]
    n_total = n_first + n_second
    return group_gini_index(first) * n_first / n_total + \
           group_gini_index(second) * n_second / n_total

## 02. Split

In [4]:
def find_split(X: np.array, y: np.array):
    n_samples, n_features = X.shape
    split_feature, split_value, best_gini, X_left, X_right, y_left, y_right = \
        None, None, None, None, None, None, None
    
    for feature_idx in range(n_features):
        order = X[:, feature_idx].argsort()
        X_sorted = X[order]
        y_sorted = y[order]
        
        for sample_idx in range(1, n_samples):
            y_left_, y_right_ = np.split(y_sorted, [sample_idx])
            gini = groups_gini_index(y_left_, y_right_)
            
            if best_gini == None or gini < best_gini:
                best_gini = gini
                split_feature = feature_idx
                split_value = (X_sorted[sample_idx, feature_idx] + X_sorted[sample_idx-1, feature_idx])/2
                y_left, y_right = y_left_, y_right_
                X_left, X_right = np.split(X_sorted, [sample_idx])
            
    return split_feature, split_value, X_left, X_right, y_left, y_right

## 03. Tree structure

In [5]:
class Node():
    def __init__(self, assigned_label = None, split_feature = None, split_value = None):
        self.assigned_label = assigned_label
        self.split_feature = split_feature
        self.split_value = split_value
        self.left_child = None
        self.right_child = None

In [14]:
class TreeClassifier():
    def __init__(self, max_depth: int, min_samples_split: int):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        
    def fit(self, X, y):
        self.root = Node()
        self.__split(X, y, self.root, 1)
        
    def predict(self, X):
        predictions = np.empty(X.shape[0])
        for idx, x in enumerate(X):
            predictions[idx] = self.__single_example_prdiction(x)
        return predictions
        
    def __split(self, X: np.array, y: np.array, node: Node, depth: int):
        n_samples = y.shape[0]
        
        if n_samples <= self.min_samples_split or depth >= self.max_depth:
            unique, counts = np.unique(y, return_counts=True)
            node.assigned_label = unique[np.argmax(counts)]
            return
        
        node.split_feature, node.split_value, X_left, X_right, y_left, y_right = find_split(X, y)
        node.left_child, node.right_child = Node(), Node()
        
        self.__split(X_left, y_left, node.left_child, depth + 1)
        self.__split(X_right, y_right, node.right_child, depth + 1)
        
    def __single_example_prdiction(self, x: np.array):
        node = self.root
        while node.assigned_label == None:
            if x[node.split_feature] > node.split_value:
                node = node.right_child
            else:
                node = node.left_child
        return node.assigned_label

## 04. Test using 'Iris' data set

### 04.00.  Imports

In [32]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### 04.01. Settings

In [30]:
FEATURES_COLUMN_NAMES = [
    "sepal_length", 
    "sepal_width", 
    "petal_length", 
    "petal_width"
]

LABELS_COLUMN_NAMES = [
    "species"
]

COLUMN_NAMES = FEATURES_COLUMN_NAMES + \
    LABELS_COLUMN_NAMES

DATA_SET_PATH = "../data_sets/iris.csv"

### 04.02. Load

In [16]:
df = pd.read_csv(DATA_SET_PATH, names=COLUMN_NAMES)
X = df[FEATURES_COLUMN_NAMES].values
y = df[LABELS_COLUMN_NAMES].values

### 04.03. Feature engineering

In [17]:
le = LabelEncoder()
y = le.fit_transform(y.ravel())

### 04.04. Data split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

### 04.05. Train

In [31]:
tree_classifier = TreeClassifier(max_depth=10, min_samples_split=2)
tree_classifier.fit(X_train, y_train)

### 04.06. Predict

In [34]:
y_pred = tree_classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {0:.2f}%".format(acc * 100))

Accuracy: 96.67%
